In [3]:
#main shuvo software download , resize , transfrom , check similar 
import tkinter as tk
import os
import requests
from bs4 import BeautifulSoup
from PIL import Image
import imagehash
import shutil
from tkinter.filedialog import askdirectory
from PIL import Image, ImageEnhance

def download_images(url, size, extension):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    img_tags = soup.find_all('img')
    if not os.path.exists('images'):
        os.makedirs('images')
    counter = 1
    width, height = size.split('*')  # Extract width and height values
    for img in img_tags:
        img_url = img['src']
        if img_url.startswith('http'):
            img_response = requests.get(img_url)
            img_data = img_response.content
            img_filename = os.path.join('images', f"{counter}.{extension}")
            with open(img_filename, 'wb') as f:
                f.write(img_data)
            resize_image(img_filename, int(width), int(height))  # Resize and convert the downloaded image
            print(f"Downloaded and resized: {img_filename}")
            counter += 1

def resize_image(img_path, width, height):
    image = Image.open(img_path)
    image = image.convert("RGB")  # Convert the image to RGB mode
    resized_image = image.resize((width, height), Image.ANTIALIAS)
    resized_image.save(img_path, "JPEG")  # Save the resized image as JPEG

def open_download_window():
    download_window = tk.Toplevel(root)
    download_window.title("Download Image")
    download_window.geometry("400x300")
    download_window.configure(bg="hotpink")  # Set window background color to hot pink
    url_label = tk.Label(download_window, text="URL:")
    url_label.pack(pady=10)
    url_entry = tk.Entry(download_window, width=40)
    url_entry.pack(pady=10)

    size_label = tk.Label(download_window, text="Image Size (width*height):")
    size_label.pack(pady=10)
    size_entry = tk.Entry(download_window, width=40)
    size_entry.pack(pady=10)

    ext_label = tk.Label(download_window, text="Image Extension:")
    ext_label.pack(pady=10)
    ext_entry = tk.Entry(download_window, width=40)
    ext_entry.pack(pady=10)

    def on_download():
        url = url_entry.get()
        size = size_entry.get()
        extension = ext_entry.get()
        download_images(url, size, extension)

    download_button = tk.Button(download_window, text="Download", command=on_download, bg="green", fg="white")
    download_button.pack(pady=10)

    exit_button = tk.Button(download_window, text="Exit", command=download_window.destroy, bg="red", fg="white")
    exit_button.pack(pady=5)


def open_remove_window():
    remove_window = tk.Toplevel(root)
    remove_window.title("Remove Similar Images")
    remove_window.geometry("400x200")
    remove_window.configure(bg="hotpink")  # Set window background color to hot pink

    label = tk.Label(remove_window, text="Enter Folders' Paths (separated by ';'):")
    label.pack(pady=10)

    folder_entry = tk.Entry(remove_window, width=40)
    folder_entry.pack(pady=5)

    def submit_folders():
        folders = folder_entry.get().split(';')
        folders = [folder.strip() for folder in folders]  # Remove leading/trailing whitespaces
        delete_similar_images(folders)

    submit_button = tk.Button(remove_window, text="Submit", command=submit_folders, bg="green", fg="white")
    submit_button.pack(pady=10)

    exit_button = tk.Button(remove_window, text="Exit", command=remove_window.destroy, bg="red", fg="white")
    exit_button.pack(pady=5)


def delete_similar_images(folders):
    images_hashes = {}

    for folder in folders:
        images = [file for file in os.listdir(folder) if file.endswith('.jpg') or file.endswith('.png')]
        for image in images:
            image_path = os.path.join(folder, image)
            hash_value = compute_image_hash(image_path)
            if hash_value in images_hashes:
                images_hashes[hash_value].append(image_path)
            else:
                images_hashes[hash_value] = [image_path]

    for hash_value, images in images_hashes.items():
        if len(images) > 1:
            for i in range(1, len(images)):
                os.remove(images[i])
                print(f"Deleted similar image: {images[i]}")

    print("Done!")


def compute_image_hash(image_path):
    with Image.open(image_path) as img:
        hash_value = imagehash.average_hash(img)
    return hash_value


def transform_images():
    # Ask for the folder path containing the images
    folder_path = askdirectory(title="Select Folder Containing Images")
    if not folder_path:
        print("Folder path not provided. Exiting...")
        return

    # Ask for the output folder path
    output_folder = askdirectory(title="Select Output Folder")
    if not output_folder:
        print("Output folder path not provided. Exiting...")
        return

    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Iterate over each file in the folder
    for filename in os.listdir(folder_path):
        # Check if the file is an image
        if filename.endswith(".jpg") or filename.endswith(".png"):
            # Load the image
            image_path = os.path.join(folder_path, filename)
            image = Image.open(image_path)

            # Image rotation
            angle = 45
            rotated_image = image.rotate(angle)

            # Image flipping
            flipped_image_horizontal = image.transpose(Image.FLIP_LEFT_RIGHT)
            flipped_image_vertical = image.transpose(Image.FLIP_TOP_BOTTOM)

            # Image shifting
            shift_x = 50
            shift_y = 50
            shifted_image = image.transform(image.size, Image.AFFINE, (1, 0, shift_x, 0, 1, shift_y))

            # Image zooming
            zoom_factor = 1.5
            zoomed_image = image.resize((int(image.width * zoom_factor), int(image.height * zoom_factor)))

            # Image shearing
            shear_factor = 0.5
            sheared_image = image.transform(image.size, Image.AFFINE, (1, shear_factor, 0, 0, 1, 0))

            # Image scaling
            scale_factor = 0.5
            scaled_image = image.resize((int(image.width * scale_factor), int(image.height * scale_factor)))

            # Image brightness adjustment
            brightness_factor = 0.5
            enhancer = ImageEnhance.Brightness(image)
            brightened_image = enhancer.enhance(brightness_factor)

            # Image cropping
            left = 100
            top = 100
            right = 300
            bottom = 300
            cropped_image = image.crop((left, top, right, bottom))

            # Save the transformed images
            rotated_image.save(os.path.join(output_folder, "rotated_" + filename))
            flipped_image_horizontal.save(os.path.join(output_folder, "flipped_horizontal_" + filename))
            flipped_image_vertical.save(os.path.join(output_folder, "flipped_vertical_" + filename))
            shifted_image.save(os.path.join(output_folder, "shifted_" + filename))
            zoomed_image.save(os.path.join(output_folder, "zoomed_" + filename))
            sheared_image.save(os.path.join(output_folder, "sheared_" + filename))
            scaled_image.save(os.path.join(output_folder, "scaled_" + filename))
            brightened_image.save(os.path.join(output_folder, "brightened_" + filename))
            cropped_image.save(os.path.join(output_folder, "cropped_" + filename))

    print("Transformations complete!")

# Create the GUI window
root = tk.Tk()
root.title("Image Transformation")
root.geometry("400x400")

# Create a button for downloading images
#download_button = tk.Button(root, text="Download Image", command=open_download_window, bg="blue", fg="white")
#download_button.pack(pady=10)

# Create a button for removing similar images
remove_button = tk.Button(root, text="Remove Similar Images", command=open_remove_window, bg="orange", fg="white")
remove_button.pack(pady=10)

# Create a button for transforming images
transform_button = tk.Button(root, text="Transform Images", command=transform_images, bg="green", fg="white")
transform_button.pack(pady=10)

# Create an exit button
exit_button = tk.Button(root, text="Exit", command=root.destroy, bg="red", fg="white")
exit_button.pack(pady=5)


# Run the GUI window's main loop
root.mainloop()
